In [ ]:
# import urllib.request
# from bs4 import BeautifulSoup
# import csv
# import pandas as pd
# import numpy as np
# import requests 
# import html5lib
# import urllib3

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

In [2]:
os.environ['PATH'] += r"/Users/marcolee/Desktop/chromedriver_mac_arm64"
driver = webdriver.Chrome()
url = "https://chooseyourstory.com/Stories/Sci-Fi.aspx"
driver.get(url)
original_window = driver.current_window_handle
driver.implicitly_wait(5)

In [3]:
def switch_window(driver,original_window):
    for window in driver.window_handles:
        if window != original_window:
            driver.switch_to.window(window)
            break

In [4]:
#get description and click play button
def start_story(driver,original_window):
    #content = driver.find_element(By.CLASS_NAME,'storygame-description').text
    play_button = driver.find_element(By.CLASS_NAME,'storygame-play-button')
    play_button.click()
    switch_window(driver,original_window)
    

In [16]:
# get the story after play button
def tree_story(driver,stories={},depth=0): #driver, dict --> text, dict[title]=story
    if depth >= 7:
        return stories, "END"
    try:
        cur_title = driver.find_element(By.ID,'_storyTitle').text
    except:
        return stories, "END"
    if 'comment' in cur_title.lower():
        return stories,"END"
    content = driver.find_element(By.XPATH,"/html/body/div[3]/div[1]").text + "\n"
    if not stories:
        flag = True
    else:
        flag = False
    if cur_title not in stories:
        stories[cur_title] = content
    # else:
        #     return stories, cur_title
    try:
        list = driver.find_elements(By.TAG_NAME,'ul')
    except:
        return stories,'END'
        #buttons = [link.text for link in list]
    links = list[1].find_elements(By.TAG_NAME,'a')
    tree = {}
    for i in range(len(links)):
        try:
            list = driver.find_elements(By.TAG_NAME,'ul')[1]
            link = list.find_elements(By.TAG_NAME,'a')[i]
        except IndexError:
            return stories,tree
            #print(link.text)
        title = link.text
        if 'end' in title.lower():
            tree = "END"
            break
        elif title in stories.keys():
            tree[title] = "END"
            continue
        link.click()
        stories, subtree = tree_story(driver,stories,depth=depth+1)
            # content += tmp_content
        tree[title] = subtree

    #go back
    list = driver.find_element(By.TAG_NAME,'ul')
    back = list.find_element(By.TAG_NAME,'a')
    back.click()
    if flag:
        return stories, {cur_title:tree}
    return stories, tree


In [7]:
# making a list of stories
table = driver.find_element(By.ID,"MainContentPlaceHolder_StoriesGridView")
stories = table.find_elements(By.TAG_NAME,'tr')
links = []
titles = []
for story in stories[1:]:
    #new_driver = copy.copy(driver)
    link = story.find_element(By.TAG_NAME,'a')
    links.append(link.get_attribute("href"))
    titles.append(link.text)
    #switch_window(driver,original_window)
    #start_story(driver,original_window)
    
#links

In [8]:
def initialize_driver(url):
    driver = webdriver.Chrome()
    driver.get(url)
    original_window = driver.current_window_handle
    #driver.implicitly_wait(5)
    return driver,original_window

In [12]:
len(links)

58

In [12]:
links.pop(1)
titles.pop(1)

'2205'

In [11]:
links = links[52:]
titles = titles[52:]

In [14]:
scifi=[]

In [26]:
len(scifi)

53

In [23]:
cur_driver.quit()

In [ ]:
#in the fields
# tales of the solar schism

In [25]:
while len(links) != 0:
    link = links.pop(0)
    title = titles.pop(0)
    cur_driver,origin_window = initialize_driver(link)
    start_story(cur_driver,origin_window)
    stories, indexes = tree_story(cur_driver,stories={})
    scifi.append({'title':title,'stories':stories,'tree':indexes})
    cur_driver.quit()

In [ ]:

for link,title in zip(links,titles):
    cur_driver,origin_window = initialize_driver(link)
    start_story(cur_driver,origin_window)
    stories, indexes = tree_story(cur_driver,stories={})
    scifi.append({'title':title,'stories':stories,'tree':indexes})
    cur_driver.quit()
    

In [42]:
cur_driver.quit()
driver.quit()

In [28]:
import json

In [29]:
scifi[0].keys()

dict_keys(['title', 'stories', 'tree'])

In [47]:
with open('scifi_text.json','w') as file:
    json.dump(scifi3,file)

In [33]:
import pandas as pd

In [46]:
scifi2 = pd.read_json('scifi_text2.json')
scifi2

,title,stories,tree
0,>DEVOTION,{'ARENA DAY': 'The gate opens. The assembled s...,{'ARENA DAY': {'>CIRCLE': {'>REPAIR': {'>LOYAL...
1,A Metric Measurment of Death,{'Born free': 'You wake up to a throbbing head...,"{'Born free': {'The Decision': {'Gary Jerome, ..."
2,A UFO at School,"{'A UFO at School': ' ', 'Math Class': 'It is ...",{'A UFO at School': {'': {'Gavin stays quiet a...
3,A.M.I.E. (1.1),{'Welcome Dr.Raymond': ' You wake up to a ...,{'Welcome Dr.Raymond': {'go get a shower': {}}}
4,Achilles,{'The First Page': 'You are General Achilles. ...,{'The First Page': {'Garavan': {'Back': {'Gara...
...,...,...,...
96,Utopia Planitia,{'A Book of Many Fates': 'UTOPIA PLANITIA Bi...,{'A Book of Many Fates': {'Next page.': {'Read...
97,WARCHIMP,{'Beginning': 'The days when the gods walked t...,{'Beginning': {'CAMP': {'TALK TO DEATH MASK AN...
98,Who STILL wants to be a Supervillian?,"{'Does it hurt?': '""You know what I love about...",{'Does it hurt?': {'It Hurts': {'Why are we he...
99,Who wants to be a Supervillain?,"{'The Beginning': 'Today is your first day, yo...",{'The Beginning': 'END'}


In [ ]:
with open('scifi_text.json') as f:
    scifi3 = json.load(f)
scifi3

In [ ]:
scifi3.extend(scifi)
scifi3

In [44]:
len(scifi3)

101

In [38]:
for i in scifi2.iloc():
    print(i)
    break

title                                              >DEVOTION
stories    {'ARENA DAY': 'The gate opens. The assembled s...
tree       {'ARENA DAY': {'>CIRCLE': {'>REPAIR': {'>LOYAL...
Name: 0, dtype: object


In [7]:
len(scifi2['stories'][0]['ARENA DAY'])

482

In [ ]:
l = scifi2['stories'][0].copy()
l.update(scifi2['stories'][5])
l

In [15]:
with open('scifi_stories.json','w') as file:
    json.dump(l,file)

In [36]:
scifi3 = scifi2.iloc[0][1].keys()
print(scifi3)

dict_keys(['ARENA DAY', '>CIRCLE', '>REPAIR', '>LOYALTY', '>REBELLION', '>STRIKE', '>RESPECT', '>TRIBUTE', '>LATER', '>SACRIFICE', '>HARVEST', '>SCORN'])
